In [1]:
%%configure -f 
{"executorMemory":"200G"}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1683391505311_0001,pyspark,busy,Link,Link,None,


In [2]:
%%configure -f
{
  "conf": {
    "spark.sql.files.maxPartitionBytes": "1073741824"
  }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1683391505311_0001,pyspark,busy,Link,Link,None,


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import types as T
import pyspark.sql.functions as F
from pyspark.sql.functions import lit
from pyspark.sql.types import IntegerType,BooleanType,DateType,LongType,DoubleType, StringType
from pyspark.sql.functions import row_number,lit,col,when, concat_ws
from pyspark.sql.window import Window
import time
spark = SparkSession.builder \
    .master("yarn") \
    .appName("Wellmark-UI") \
    .getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1683391505311_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
npi_table = spark.read.parquet("s3://zigna-nsa-data-staging/metadata/Q2-2023/helper-tables/apr-wellmark-hosp-ancillary/april_wellmark_npi_table.parquet")

In [ ]:
npi_table.printSchema()
len(npi_table.columns)

In [ ]:
npi_table = npi_table.withColumnRenamed('provider_business_practice_location_address_city_name', 'provider_city_name').withColumnRenamed(
    'provider_business_practice_location_address_state_name', 'provider_state_name').withColumnRenamed('postal_code', 'provider_postal_code').withColumnRenamed('npi', 'master_npi')
npi_table = npi_table.withColumn("primary_city",npi_table.primary_city.cast(StringType()))

In [ ]:
npi_table=npi_table.orderBy('provider_name')

In [ ]:
npi_table.groupBy('region').count().show()

In [ ]:
npi_table.distinct().count()

In [ ]:
npi_table.groupBy('provider_type').count().show()

In [ ]:
data = spark.read.parquet("s3://zigna-nsa-payer-data-parquet-staging/Q2-2023/wellmark/ui-table/wellmark/flat-table/apr-sanford/")

In [ ]:
data.printSchema()
len(data.columns)

In [ ]:
data[['reporting_entity_name']].distinct().show()

In [ ]:
data[['reporting_entity_type']].distinct().show()

In [ ]:
data[['service_code']].distinct().show()

In [ ]:
data = data.filter(data.negotiated_rate > 0.0)
data = data.filter(~((data.negotiated_type == 'percentage') & (data.negotiated_rate > 250.0)))
data = data.withColumn("negotiated_rate",data.negotiated_rate.cast(DoubleType()))
data = data.withColumn("npi",data.npi.cast(StringType()))

In [ ]:
data.count()

In [ ]:
data1 = npi_table.join(data,npi_table.master_npi == data.npi, 'inner')
data1.printSchema()
len(data1.columns)

In [ ]:
data1 = data1.drop('grouping').withColumnRenamed('provider_specialty_group','grouping').drop('classification').withColumnRenamed('provider_specialty_description','classification').drop('display_name').withColumnRenamed('provider_credential','display_name')

In [ ]:
data1[['grouping']].distinct().show()

In [ ]:
data1[['classification']].distinct().show()

In [ ]:
data1[['display_name']].distinct().show()

In [ ]:
data1.count()

In [ ]:
data2 = data1.filter(data1.provider_type.isin(['Hospital','Ancillary']))

In [ ]:
data2 = data2.distinct()
data2.count()

In [ ]:
ui_table = data2

In [ ]:
data2.show()

In [ ]:
if str(ui_table.schema["date"].dataType) == 'StringType' and str(ui_table.schema["npi"].dataType) == 'StringType' and str(ui_table.schema["state"].dataType) == 'StringType' and str(ui_table.schema["r_billing_code_type"].dataType) == 'StringType' and str(ui_table.schema["r_billing_code"].dataType) == 'StringType' and str(ui_table.schema["negotiated_rate"].dataType) == 'DoubleType' and str(ui_table.schema["partition_column"].dataType) == 'StringType' and len(ui_table.columns) == 34 and str(ui_table.schema["reporting_entity_name"].dataType) == 'StringType' and str(ui_table.schema["billing_code"].dataType) == 'StringType' and str(ui_table.schema["billing_code_type"].dataType) == 'StringType' and str(ui_table.schema["billing_code_type_version"].dataType) == 'StringType' and str(ui_table.schema["billing_class"].dataType) == 'StringType' and str(ui_table.schema["billing_code_modifier"].dataType) == 'StringType' and str(ui_table.schema["negotiation_arrangement"].dataType) == 'StringType' and str(ui_table.schema["negotiated_type"].dataType) == 'StringType' and str(ui_table.schema["service_code"].dataType) == 'StringType' and str(ui_table.schema["additional_information"].dataType) == 'StringType' and str(ui_table.schema["level1_description"].dataType) == 'StringType' and str(ui_table.schema["description"].dataType) == 'StringType' and str(ui_table.schema["provider_name"].dataType) == 'StringType' and str(ui_table.schema["provider_city_name"].dataType) == 'StringType' and str(ui_table.schema["provider_state_name"].dataType) == 'StringType' and str(ui_table.schema["provider_postal_code"].dataType) == 'StringType' and str(ui_table.schema["county"].dataType) == 'StringType' and str(ui_table.schema["region"].dataType) == 'StringType' and str(ui_table.schema["provider_type"].dataType) == 'StringType' and str(ui_table.schema["grouping"].dataType) == 'StringType' and str(ui_table.schema["classification"].dataType) == 'StringType' and str(ui_table.schema["display_name"].dataType) == 'StringType' and str(ui_table.schema["plan_name"].dataType) == 'StringType' and str(ui_table.schema["reporting_entity_type"].dataType) == 'StringType' and str(ui_table.schema["level4_description"].dataType) == 'StringType' and str(ui_table.schema["zip"].dataType) == 'StringType' and str(ui_table.schema["primary_city"].dataType) == 'StringType':
    data2.write.partitionBy('partition_column').mode("append").parquet("s3://zigna-nsa-payer-data-parquet-staging/Q2-2023/wellmark/ui-table/wellmark/partitioned_parquet/hospital_ancillary/datazigna_final_data_model_ui_partitioned_v1/")
    print('true')
else:
    print('error')

In [7]:
df = spark.read.options(header = True).csv("s3://zignaai-deidentified-claimsdata/pi-claims-201901-202203-deidentified.txt", sep="|")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- CLAIM_CONTROL_INFORMATION: string (nullable = true)
 |-- CLAIM_ID: string (nullable = true)
 |-- CLAIM_RECORD_LOCATOR: string (nullable = true)
 |-- ADJUDICATION_ID: string (nullable = true)
 |-- ADJUDICATION_RECORD_LOCATOR: string (nullable = true)
 |-- ADJUDICATION_CREATED_AT: string (nullable = true)
 |-- ADJUDICATION_FINALIZED_AT: string (nullable = true)
 |-- CLAIM_FAMILY_ID: string (nullable = true)
 |-- CLAIM_TYPE: string (nullable = true)
 |-- CLAIM_FREQUENCY: string (nullable = true)
 |-- CLAIM_SUBMISSION_TYPE: string (nullable = true)
 |-- AUTO_ADJUDICATED: string (nullable = true)
 |-- ADJUDICATOR_ID: string (nullable = true)
 |-- OON_INDICATOR: string (nullable = true)
 |-- DENIED_INDICATOR: string (nullable = true)
 |-- ADJUDICATION_STATUS: string (nullable = true)
 |-- ADJUDICATION_SEQUENCE_NUMBER: string (nullable = true)
 |-- PAYER_CONTROL_NUMBER: string (nullable = true)
 |-- REMIT_STATUS: string (nullable = true)
 |-- PAYMENT_TRACKING_NUMBER: string (nullabl

In [9]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+--------------------+--------------------+--------------------+---------------------------+-----------------------+-------------------------+--------------------+-------------+---------------+---------------------+----------------+--------------------+-------------+----------------+-------------------+----------------------------+--------------------+------------+-----------------------+--------------------+----------------+--------------------+-------------------+------------------+-------------------+-------------------------+--------------+------------+----------------------+-----------------------+--------------+------------+---------------------+----------------------------+--------------------+------------------+--------------------+---------------------+------------------+-----------------+---------+----------------+-------------+----------+------+--------------------+--------------------+--------------------+-------------------+--------------------+--

In [11]:
from pyspark.sql.functions import *

df = df.withColumn("FIRST_SERVICE_DATE",to_date("FIRST_SERVICE_DATE"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df[['FIRST_SERVICE_DATE']].show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|FIRST_SERVICE_DATE|
+------------------+
|        2021-09-23|
|        2021-12-02|
|        2021-03-29|
|        2020-02-24|
|        2021-08-09|
|        2021-01-12|
|        2021-04-27|
|        2021-04-19|
|        2022-02-24|
|        2019-02-21|
|        2022-02-02|
|        2021-09-22|
|        2021-10-01|
|        2020-10-13|
|        2019-02-15|
|        2020-02-01|
|        2021-09-03|
|        2021-03-01|
|        2021-07-12|
|        2020-09-28|
+------------------+
only showing top 20 rows